# 데이터 사전

[독립변수]
    
{  
- 공통사항
- subject_id: 사용자 ID
- timestamp: 타임스탬프  
}
- mACSStatus   
  m_charging: 충전여부 (0, 1)  
  0: 비충전  
  1: 충전중  

- mActivity  
  m_activity: 활동 유형 (0~8)  
   0: IN_VEHICLE - 차량 내에 있음 (자동차, 버스 등 교통수단을 이용 중인 상태)  
   1: ON_BICYCLE - 자전거 타는 중 (자전거를 타고 이동 중인 상태)  
   2: ON_FOOT - 걷는 중 (보행 중인 상태로, 걷기와 달리기를 포함하는 일반적인 활동)  
   3: STILL - 정지 상태 (움직임이 없는 상태, 앉아있거나 서있는 등)  
   4: UNKNOWN - 알 수 없음 (활동을 식별할 수 없는 상태)  
   5: TILTING - 기울어짐 (기기가 기울어지거나 방향이 변하는 상태)  
   7: WALKING - 걷는 중 (보행 중인 상태, ON_FOOT보다 더 구체적인 걷기 활동)  
   8: RUNNING - 달리는 중 (달리기 활동을 하는 상태)  

- m_Ambience  
  m_ambience: 주변 소리
  - 주변 소리 레이블과 각 레이블의 확률 목록

- mBle  
  mble: 블루투스 장치
  - 블루투스 장치 주소, 디바이스 분류, rssi 목록

- mGps  
  m_gps: 위치 정보
  - (고도, 위도, 경도, 속도)

- mLight  
  m_light
  - 주변광 (lx 단위)

- mScreenStatus  
  m_screen_use: 화면 사용 여부  
   0: 비사용,   
   1: 화면 사용 중  

- mUsageStats  
  m_usage_stats: 앱 사용
  - 앱 이름과 해당 사용 시간(밀리초 단위)

- mWifi  
  m_wifi
  - 객체 기지국 ID(bssid) 및 rssi 목록

- wHr  
  heart_rate
  - 심박수 기록

- wLight   
  w_light  
  - 주변광 (lx 단위)  

- wPedo  
  burned_calories: 소모 칼로리
  - 칼로리 수
  distance: 거리
  - 거리(미터)
  speed: 속도
  - 속도(km/h)
  step: 걸음
  - 걸음 수
  step_frequency: 보속
  - 분당 걸음 속도

[종속변수]
- subject_id: 사용자 ID  
- sleep_date: 수면 기록 날짜  
- lifelog_date: 일일 활동 날짜 (sleep_date 전날)  
- Q1: 기상 직후 전반적인 수면 품질  
   0: 개인 평균 미만,  
   1: 개인 평균 이상
- Q2: 수면 직전 피로도  
   0: 높은 수준의 피로도,  
   1: 낮은 수준의 피로도
- Q3: 수면 직전 스트레스 수준  
   0: 높은 수준의 스트레스,  
   1: 낮은 수준의 스트레스
- S1: 총 수면 시간 지침 준수 여부(TST)  
   0: 비적절,  
   1: 적절할 수 있음,  
   2: 적절  
- S2: 수면 효율성 지침 준수 여부(SE)  
   0: 부적절  
   1: 적절   
- S3: 수면 시작 지연 시간(SOL, SL)  
   0: 부적절  
   1: 적절  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import glob
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import ast
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from scipy.stats import entropy
from scipy.stats import iqr
import collections

from tqdm import tqdm
from collections import defaultdict
from collections import Counter
from itertools import combinations

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from pytorch_tabnet.tab_network import TabNetEncoder

In [ ]:
seed = 810

def set_seed(sd):
    random.seed(sd)
    np.random.seed(sd)
    os.environ['PYTHONHASHSEED'] = str(sd)
    random.seed(sd)
    np.random.seed(sd)
    torch.manual_seed(sd)
    torch.cuda.manual_seed_all(sd)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(seed)

# 데이터 불러오기 및 EDA

In [ ]:
data_dir = '/content/drive/MyDrive/1데이콘/ETRI_휴먼이해_인공지능/ETRI_lifelog_dataset/ch2025_data_items'

In [ ]:
dataframe = {}
for file_name in sorted(os.listdir(data_dir)):
  if not file_name.endswith('.parquet'):
    continue
  sensor_name = file_name.replace('ch2025_', '').replace('.parquet', '')

  df = pd.read_parquet(os.path.join(data_dir, file_name))
  dataframe[sensor_name] = df
  print(f'Loaded {sensor_name}, shape = {dataframe[sensor_name].shape}')

Loaded mACStatus, shape = (939896, 3)
Loaded mActivity, shape = (961062, 3)
Loaded mAmbience, shape = (476577, 3)
Loaded mBle, shape = (21830, 3)
Loaded mGps, shape = (800611, 3)
Loaded mLight, shape = (96258, 3)
Loaded mScreenStatus, shape = (939653, 3)
Loaded mUsageStats, shape = (45197, 3)
Loaded mWifi, shape = (76336, 3)
Loaded wHr, shape = (382918, 3)
Loaded wLight, shape = (633741, 3)
Loaded wPedo, shape = (748100, 9)


In [ ]:
# parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet'))

# dataframe = {}

# for file_path in parquet_files:
#     sensor_name = os.path.basename(file_path).replace('ch2025_', '').replace('.parquet', '')
#     dataframe[sensor_name] = pd.read_parquet(file_path)
#     print(f" Loaded: {sensor_name}, shape = {dataframe[sensor_name].shape}")

In [ ]:
for name, df in dataframe.items():
  print(f'== {name} ==')
  print(df.head(3), '\n')
  print(df.info(),'\n')
  print('-' * 50)

== mACStatus ==
  subject_id           timestamp  m_charging
0       id01 2024-06-26 12:03:00           0
1       id01 2024-06-26 12:04:00           0
2       id01 2024-06-26 12:05:00           0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939896 entries, 0 to 939895
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   subject_id  939896 non-null  object        
 1   timestamp   939896 non-null  datetime64[ns]
 2   m_charging  939896 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 21.5+ MB
None 

--------------------------------------------------
== mActivity ==
  subject_id           timestamp  m_activity
0       id01 2024-06-26 12:03:00           4
1       id01 2024-06-26 12:04:00           0
2       id01 2024-06-26 12:05:00           0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961062 entries, 0 to 961061
Data columns (total 3 columns):
 #   Column

In [ ]:
# 결측치 및 요약
for name, df in dataframe.items():
  print(f'== {name} ==')
  print(f'Missing Value', (df.isnull().mean() * 100).round(2).to_dict())
  num_cols = df.select_dtypes(include='number').columns
  if len(num_cols) > 0:
    print(f'Summary Statistic', '\n', df[num_cols].describe().loc[['mean','std','min','max']])
  print()

== mACStatus ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_charging': 0.0}
Summary Statistic 
       m_charging
mean    0.291378
std     0.454397
min     0.000000
max     1.000000

== mActivity ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_activity': 0.0}
Summary Statistic 
       m_activity
mean    2.986991
std     1.132508
min     0.000000
max     8.000000

== mAmbience ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_ambience': 0.0}

== mBle ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_ble': 0.0}

== mGps ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_gps': 0.0}

== mLight ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_light': 0.0}
Summary Statistic 
             m_light
mean     185.176751
std     1939.970572
min        0.000000
max   334306.000000

== mScreenStatus ==
Missing Value {'subject_id': 0.0, 'timestamp': 0.0, 'm_screen_use': 0.0}
Summary Statistic 
       m_screen_use
mean      0.228389
std       0.41979

In [ ]:
for name, df in dataframe.items():
  if name.endswith('Light'):
    num_col = df.select_dtypes(include='number').columns
    for col in num_col:
      print(col)
      q1 = df[col].quantile(0.25)
      q3 = df[col].quantile(0.75)
      iqr = q3 - q1
      lower = q1 - (1.5 * iqr)
      upper = q3 + (1.5 * iqr)
      outliers = df[(df[col] < lower) | (df[col] > upper)]
      if not outliers.empty:
        print(f'Lower: {lower}, Upper: {upper}, Outlier length: {len(outliers)}, Outlier ratio: {(len(outliers) / len(df)) * 100:.2f}%, \n')
      df.loc[df[col] > upper, col] = 100000.0
  #   if len(col) > 0:
  #     sns.boxplot(y=df[col])
  #     plt.title(f'Boxplot of {name}')
  #     plt.show()
  # print()

m_light
Lower: -235.5, Upper: 392.5, Outlier length: 11638, Outlier ratio: 12.09%, 

w_light
Lower: -198.0, Upper: 330.0, Outlier length: 71751, Outlier ratio: 11.32%, 



In [ ]:
# 초 단위 제거
for name, df in dataframe.items():
  df['timestamp'] =  df['timestamp'].dt.floor('min')
  df['date'] = df['timestamp'].dt.date
  dataframe[name] = df

In [ ]:
# 변수마다 데이터프레임화
for key, df in dataframe.items():
  globals()[f'{key}_df'] = df

In [ ]:
# dataframe = {
#     'mACStatus': (mACStatus_df, 'timestamp'),
#     'mActivity': (mActivity_df, 'timestamp'),
#     'mAmbience': (mAmbience_df, 'timestamp'),
#     'mBle': (mBle_df, 'timestamp'),
#     'mGps': (mGps_df, 'timestamp'),
#     'mLight': (mLight_df, 'timestamp'),
#     'mScreenStatus': (mScreenStatus_df, 'timestamp'),
#     'mUsageStats': (mUsageStats_df, 'timestamp'),
#     'mWifi': (mWifi_df, 'timestamp'),
#     'wHr': (wHr_df, 'timestamp'),
#     'wLight': (wLight_df, 'timestamp'),
#     'wPedo': (wPedo_df, 'timestamp'),
# }

# 변수별 전처리

In [ ]:
# def explode(df):
#   col = [col for col in df.columns if col.startswith('m_')]
#   assert len(col) == 1
#   col = col[0]

#   explode_df = df.explode(col, ignore_index=True)
#   detail_df = pd.json_normalize(explode_df[col])
#   result_df = pd.concat([explode_df.drop(columns = col), detail_df], axis=1)

#   return result_df

In [ ]:
def sleep_lifelog(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['sleep_date'] = np.where(
        df['timestamp'].dt.hour < 7,
        df['timestamp'].dt.date,
        df['timestamp'].dt.date + pd.Timedelta(days=1)
    )
    df['lifelog_date'] = pd.to_datetime(df['sleep_date']) - pd.Timedelta(days=1)

    return df

In [ ]:
def find_col(df):
    col = [col for col in df.columns if col.startswith('m_') or col.startswith('w_') or col.startswith('he')]
    assert len(col) == 1
    col = col[0]

    return col

In [ ]:
def cut_time(hour):
    if 22 <= hour or 0 <= hour < 6:
        return 'S1_S2'

In [ ]:
time_list = ['step_list']

def view_corr(df):
    for tl in time_list:
      numeric_cols = df.select_dtypes(include='number').columns
      tl_cols = [col for col in numeric_cols if col.startswith(tl)]
      corr_df = df[tl_cols]
      print(f'{tl}_corr_list')
      print(pd.DataFrame(corr_df.corr().drop_duplicates()))
      print('-'*50, '\n')

### mACStatus

In [ ]:
def preprocessing_mACStatus(dfs):
    df = sleep_lifelog(dfs)

    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

    col = find_col(df)

    results = []
    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        status = part_group[col].values  # 0/1 상태

        # 충전상태 길이
        charging_period = sum(status) / len(status)
        row[f'{part}@charging_period'] = charging_period

        # 충전 상태 평균
        # ratio_charging = status.mean()
        # row[f'{part}@charging_ratio'] = ratio_charging

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum() / len(status)
        row[f'{part}@charging_transitions'] = transitions

        # 충전 패턴 분포
        _, counts = np.unique(status, return_counts=True)
        prob = counts / counts.sum()
        ent = entropy(prob) # 1에 가까울수록 충전 상태가 자주 바뀜
        row[f'{part}@charging_entropy'] = ent

        # 연속된 1 상태 길이
        length = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                length.append(current_len)
                current_len = 0
        if current_len > 0:
            length.append(current_len)

        # 평균 충전기간
        # avg_charging_period = (np.mean(length) / len(length)) if length else 0
        # row[f'{part}@charging_avg_period'] = avg_charging_period

        # 최대 충전기간
        # max_charging_period = (np.max(length)/ len(length)) if length else 0
        # row[f'{part}@charging_max_period'] = max_charging_period

        # 불안전성
        row[f'{part}@charging_instability'] = (1 + ent) * (1 + transitions) * (1 + charging_period)

      # 결과값 데이터프레임
      results.append(row)

    return pd.DataFrame(results).fillna(0)

pre_mACStatus_df = preprocessing_mACStatus(mACStatus_df)

# def cummean_mACStatus(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mACStatus_df = cummean_mACStatus(pre_mACStatus_df)

# def final_mACStatus(pre_mACStatus_df, cummean_mACStatus_df):
#   merge_df = pre_mACStatus_df.merge(cummean_mACStatus_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mACStatus_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

#   return final_df

# final_mACStatus_df = final_mACStatus(pre_mACStatus_df, cummean_mACStatus_df)

In [ ]:
pre_mACStatus_df

,subject_id,date,S1_S2@charging_period,S1_S2@charging_transitions,S1_S2@charging_entropy,S1_S2@charging_avg_period,S1_S2@charging_max_period,S1_S2@charging_instability
0,id01,2024-06-26,0.120253,0.008439,0.367429,14.25,19.5,1.544794
1,id01,2024-06-27,0.065957,0.004255,0.243054,31.00,31.0,1.330681
2,id01,2024-06-28,0.880435,0.008696,0.366058,101.25,202.0,2.591120
3,id01,2024-06-29,0.000000,0.000000,0.000000,0.00,0.0,1.000000
4,id01,2024-06-30,0.039130,0.008696,0.165171,4.50,7.5,1.221293
...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.791667,0.008333,0.511740,95.00,95.0,2.731105
799,id10,2024-09-23,1.000000,0.000000,0.000000,340.00,340.0,2.000000
800,id10,2024-09-24,0.260000,0.004444,0.573057,29.25,53.5,1.990861
801,id10,2024-09-25,0.440476,0.004762,0.686044,46.25,80.5,2.440272


### mActivity

In [ ]:
def preprocessing_mActivity(dfs):
  df = sleep_lifelog(dfs)

  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date

  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

  col = find_col(df)

  results = []
  for (sub, date), daily_group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':sub, 'date':date}
      for part, part_group in daily_group.groupby('time_part'):
        status = part_group[col].values

        # 활동 다양성
        activity_diversity = (part_group[col].nunique() / 8)
        row[f'{part}@activity_diversity'] = activity_diversity
        # 가장 많은 활동
        # activity_frequence = part_group[col].mode()[0] if not part_group[col].mode().empty else np.nan
        # row[f'{part}@activity_frequence'] = activity_frequence

        # 활동 유형 비율
        count = part_group[col].value_counts(normalize=True)

        # 가장 많은 활동 비율
        # row[f'{part}@activity_frequence_ratio'] = count.max()

        # 활동 다양성 분포
        ent = entropy(count)
        row[f'{part}@activity_entropy'] = ent

        # 활동 바뀐 횟수
        changes = (status[1:] != status[:-1]).sum()
        row[f'{part}@activity_change'] = changes / len(status)

        # 연속된 활동 저장
        length_activity = {}
        for j in range(9):
          length = []
          current = 0
          for val in status:
            if val == j:
              current += 1
            elif current > 0:
              length.append(current)
              current = 0
          if current > 0:
            length.append(current)
          length_activity[j] = length

        # 총 활동 수
        row[f'{part}@total_activity'] = sum(k + 1 * item
                                            for k, lst in length_activity.items()
                                            if k not in [3, 4]
                                            for item in lst) / len(length_activity)

        # # 활동 유형별 비율
        dynamic_state = [0,1,2,7,8]
        static_state = [3,5]

        # dynamic_state_cnt = np.isin(status, dynamic_state).sum()
        # row[f'{part}@dynamic_state_cnt'] = dynamic_state_cnt
        dynamic_state_ratio = sum([count.get(i,0) for i in dynamic_state])
        row[f'{part}@dynamic_state_ratio'] = dynamic_state_ratio

        # static_state_cnt = np.isin(status, static_state).sum()
        # row[f'{part}@static_state_cnt'] = static_state_cnt
        static_state_ratio = sum([count.get(i,0) for i in static_state])
        row[f'{part}@static_state_ratio'] = static_state_ratio

        # 활동 불안전성
        row[f'{part}@activity_instability'] = (1 + ent) * (1 + changes) * (1 + activity_diversity)

      results.append(row)

  return pd.DataFrame(results)

pre_mActivity_df = preprocessing_mActivity(mActivity_df)

# def cummean_mActivity(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mActivity_df = cummean_mActivity(pre_mActivity_df)

# def final_mActivity(pre_mActivity_df, cummean_mActivity_df):
#   merge_df = pre_mActivity_df.merge(cummean_mActivity_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mActivity_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mActivity_df = final_mActivity(pre_mActivity_df, cummean_mActivity_df)

In [ ]:
pre_mActivity_df

,subject_id,date,S1_S2@activity_diversity,S1_S2@activity_entropy,S1_S2@activity_change,S1_S2@total_activity,S1_S2@dynamic_state_ratio,S1_S2@static_state_ratio,S1_S2@activity_instability
0,id01,2024-06-26,0.500,0.398384,0.010549,3.333333,0.048523,0.046414,12.585453
1,id01,2024-06-27,0.375,0.506840,0.010417,11.444444,0.185417,0.814583,12.431426
2,id01,2024-06-28,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
3,id01,2024-06-29,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
4,id01,2024-06-30,0.375,0.323327,0.012500,6.444444,0.091667,0.908333,12.737021
...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.250,0.048194,0.016667,0.888889,0.008333,0.991667,3.930729
799,id10,2024-09-23,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
800,id10,2024-09-24,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000
801,id10,2024-09-25,0.125,0.000000,0.000000,0.000000,0.000000,1.000000,1.125000


### mAmbience

In [ ]:
def preprocessing_mAmbience(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': subj, 'date': date}
      for part, part_group in group.groupby('time_part'):
        all_labels_scores = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device

          sum_score = sum(float(score) for _, score in flat if isinstance(score, (int, float, str)))
          all_labels_scores.extend([
              (label, float(normalized_score))
              for label, score in flat
              if sum_score > 0
              for normalized_score in [float(score) / sum_score]
              if 0 < float(normalized_score) < 1
              ])

          if all_labels_scores:
            label_list = [label for label, _ in all_labels_scores]
            score_list = [score for _, score in all_labels_scores]
            labels, scores = zip(*all_labels_scores)
            scores = list(scores)
            max_idx = np.argmax(scores)
            min_idx = np.argmin(scores)

            # 소음 최대
            # row[f'{part}@ambience_max_label'] = labels[max_idx]

            # 소음 최소
            # row[f'{part}@ambience_min_label'] = labels[min_idx]

            range = scores[max_idx] - scores[min_idx]
            row[f'{part}@ambience_range'] = range

            # 소음 형태 고유값
            ambience_cnt = len(set(label_list)) / len(label_list)
            row[f'{part}@ambience_cnt'] = ambience_cnt

            # 소음 형태 다양성 분포
            probs_counter = Counter(label_list)
            total = sum(probs_counter.values())
            probs = [v / total for v in probs_counter.values()]
            ent = entropy(probs)
            row[f'{part}@ambience_entropy'] = ent

            # 기초 통계량
            mean = np.mean(scores) if sum(scores) > 0 else 0
            row[f'{part}@ambience_mean'] = mean
            std = np.std(scores) if sum(scores) > 0 else 0
            row[f'{part}@ambience_std'] = std
            cov = (np.std(scores) / np.mean(scores)) if sum(scores) > 0 else 0
            row[f'{part}@ambience_cov'] = cov

            # 소음 불안전성
            row[f'{part}@ambience_instability'] = (1 + ent) * (1 + ambience_cnt) * (1 + cov)

      results.append(row)

    return pd.DataFrame(results)

pre_mAmbience_df = preprocessing_mAmbience(mAmbience_df)

# def cummean_mAmbience(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mAmbience_df = cummean_mAmbience(pre_mAmbience_df)

# def final_mAmbience(pre_mAmbience_df, cummean_mAmbience_df):
#   merge_df = pre_mAmbience_df.merge(cummean_mAmbience_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mAmbience_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mAmbience_df = final_mAmbience(pre_mAmbience_df, cummean_mAmbience_df)

In [ ]:
pre_mAmbience_df

,subject_id,date,S1_S2@ambience_range,S1_S2@ambience_cnt,S1_S2@ambience_entropy,S1_S2@ambience_mean,S1_S2@ambience_std,S1_S2@ambience_cov,S1_S2@ambience_instability
0,id01,2024-06-26,1.000000,0.018349,1.777860,0.196330,0.393225,2.002873,8.494618
1,id01,2024-06-27,1.000000,0.019835,1.711205,0.198347,0.397023,2.001660,8.299531
2,id01,2024-06-28,1.000000,0.009959,1.647090,0.199170,0.399316,2.004900,8.033454
3,id01,2024-06-29,1.000000,0.004167,1.609438,0.200000,0.400000,2.000000,7.860931
4,id01,2024-06-30,1.000000,0.011618,1.657843,0.199170,0.398493,2.000767,8.068228
...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.967609,0.306667,4.641312,0.100000,0.169753,1.697527,19.884316
799,id10,2024-09-23,0.958208,0.066667,3.627382,0.100000,0.108070,1.080703,10.270086
800,id10,2024-09-24,0.964599,0.096667,4.329280,0.100000,0.171329,1.713292,15.857683
801,id10,2024-09-25,0.959353,0.098333,4.443462,0.100000,0.133683,1.336826,13.971267


### mBle

In [ ]:
device_class_labels = {'Unknown': ['0', '7936'],
                      'Computer': ['256', '284'],
                      'Phone': ['524'],
                      'Audio_Video':['1024', '1028', '1044', '1048', '1060', '1064', '1084'],
                      'Peripheral':['1280', '1344', '1408'],
                      'Wearable':['1796'],
                      'Health':['2324']}

device_labels = {
    'Unknown': device_class_labels['Unknown'],
    'Recognizable': []
}

for k, v in device_class_labels.items():
  if k != 'Unknown':
    device_labels['Recognizable'] += v

print(device_labels)

{'Unknown': ['0', '7936'], 'Recognizable': ['256', '284', '524', '1024', '1028', '1044', '1048', '1060', '1064', '1084', '1280', '1344', '1408', '1796', '2324']}


In [ ]:
def preprocessing_mBle(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):

        rssi_list = []
        address_list = []
        class_counts = {f'class_{label}_cnt': 0 for label in device_labels}

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for device in flat:

              # rssi 목록
              rssi = int(device.get('rssi'))
              rssi_list.append(rssi)

              # address 목록
              address = str(device.get('address'))
              address_list.append(address)

              # 디바이스 장치 수
              device_class = str(device.get('device_class'))
              for label, ids in device_labels.items():
                if device_class in ids:
                  class_counts[f'class_{label}_cnt'] += 1

        # rssi 고유값
        rssi_cnt = len(set(rssi_list)) / len(rssi_list)
        row[f'{part}@rssi_cnt'] = rssi_cnt

        # rssi 다양성
        rssi_counter = Counter(address_list)
        total = sum(rssi_counter.values())
        probs = [v / total for v in rssi_counter.values()]
        ent_rssi = entropy(probs)
        row[f'{part}@rssi_entropy'] = ent_rssi

        # address 고유값
        address_cnt = len(set(address_list)) / len(address_list)
        row[f'{part}@address_cnt'] = address_cnt

        # address 다양성
        probs_counter = Counter(address_list)
        total = sum(probs_counter.values())
        probs = [v / total for v in probs_counter.values()]
        ent_address = entropy(probs)
        row[f'{part}@address_entropy'] = ent_address

        # 라벨 수
        total_count = sum(class_counts.values())
        for label in device_labels:
          if total_count > 0:
            row[f'{part}@{label}_cnt'] = class_counts[f'class_{label}_cnt'] / total_count
          else:
            row[f'{part}@{label}_cnt'] = 0
          # row[f'{part}@{label}_ratio'] = (class_counts[f'class_{label}_cnt'] / total_devices if total_devices > 0 else 0)

        # 라벨 다양성
        counts = np.array(list(class_counts.values()))
        ent_label = entropy(counts)
        row[f'{part}@label_entropy'] = ent_label

        # 기초 통계
        mean = np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_mean'] = np.mean(rssi_list) if rssi_list else 0
        min = np.min(rssi_list) if rssi_list else 0
        max = np.max(rssi_list) if rssi_list else 0
        range = np.max(rssi_list) - np.min(rssi_list)
        row[f'{part}@rssi_range'] = range
        std = np.std(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_std'] = std
        cov = np.std(rssi_list) / np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_cov'] = cov

        # mble 불안전성
        row[f'{part}@mble_instability'] = ((1 + ent_rssi) * (1 + rssi_cnt) *
         (1 + ent_address) * (1 + address_cnt) *
          (1 + ent_label) * (1 + (class_counts['class_Recognizable_cnt'] if class_counts['class_Recognizable_cnt'] else 0)) *
           (1 + cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mBle_df = preprocessing_mBle(mBle_df)

# def cummean_mBle(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mBle_df = cummean_mBle(pre_mBle_df)

# def final_mBle(pre_mBle_df, cummean_mBle_df):
#   merge_df = pre_mBle_df.merge(cummean_mBle_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mBle_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'Q1' in col:
#         final_df[f'{col}'] = (merge_df[col] > merge_df[cummean_col]).astype(int)
#       elif 'Q2_Q3' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mBle_df = final_mBle(pre_mBle_df, cummean_mBle_df)

In [ ]:
pre_mBle_df

,subject_id,date,S1_S2@rssi_cnt,S1_S2@rssi_entropy,S1_S2@address_cnt,S1_S2@address_entropy,S1_S2@Unknown_cnt,S1_S2@Recognizable_cnt,S1_S2@label_entropy,S1_S2@rssi_mean,S1_S2@rssi_range,S1_S2@rssi_std,S1_S2@rssi_cov,S1_S2@mble_instability
0,id01,2024-06-26,0.489362,3.202481,0.638298,3.202481,0.872340,0.127660,0.381913,-74.553191,44.0,10.880020,-0.145936,356.019306
1,id01,2024-06-27,0.846154,2.311423,0.846154,2.311423,0.846154,0.153846,0.429323,-76.230769,31.0,8.321925,-0.109168,142.762090
2,id01,2024-06-28,0.606061,3.126627,0.787879,3.126627,0.848485,0.151515,0.425328,-74.333333,31.0,9.762844,-0.131339,363.251123
3,id01,2024-06-29,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,-72.000000,0.0,0.000000,-0.000000,4.000000
4,id01,2024-06-30,0.703704,3.193148,0.925926,3.193148,0.925926,0.074074,0.264052,-71.000000,31.0,8.041559,-0.113261,193.997415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,id10,2024-09-22,0.354839,3.220853,0.483871,3.220853,1.000000,0.000000,0.000000,-86.333333,42.0,10.072141,-0.116666,31.638024
722,id10,2024-09-23,0.800000,2.163956,0.900000,2.163956,1.000000,0.000000,0.000000,-83.200000,36.0,11.548160,-0.138800,29.484306
723,id10,2024-09-24,0.250000,3.434574,0.448718,3.434574,1.000000,0.000000,0.000000,-84.346154,44.0,10.389624,-0.123178,31.225463
724,id10,2024-09-25,0.248619,3.379794,0.392265,3.379794,1.000000,0.000000,0.000000,-84.640884,51.0,13.157704,-0.155453,28.163251


### mGps

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(coord1, coord2):
    R = 6371  # 지구 반지름 (km)
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [ ]:
def preprocessing_mGps(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        distances = []
        altitudes_list = []
        latitudes_list = []
        longitudes_list = []
        speeds_list = []
        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for entry in flat:
            altitudes_list.append(float(entry['altitude']))
            latitudes_list.append(float(entry['latitude']))
            longitudes_list.append(float(entry['longitude']))
            speeds_list.append(float(entry['speed']))

        # 중심점
        if latitudes_list and longitudes_list:
          center = (np.mean(latitudes_list), np.mean(longitudes_list))
          distances = [haversine(center, (lat, lng)) for lat, lng in zip(latitudes_list, longitudes_list)]

          # 중심점으로부터 최대 이동거리
          radius = max(distances) if distances else np.nan
          row[f'{part}@center_from_max_distance'] = radius

          # 중심점으로부터 총 이동거리
          center_total_distance = sum(distances)
          row[f'{part}@center_total_distance'] = center_total_distance

          coords = list(zip(latitudes_list, longitudes_list))
          if len(coords) >= 2:
            total_distance = sum(haversine(coords[i], coords[i+1]) for i in range(len(coords) - 1))
          else:
            total_distance = 0.0
          row[f'{part}@total_distance'] = total_distance

          # 이동경로
          path_tortuosity = ((center_total_distance / total_distance) if total_distance > 0 else 0)
          row[f'{part}@path_tortuosity'] = path_tortuosity

        # 고도 기초 통계량
        altitude_mean = np.mean(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_mean'] = altitude_mean
        altitude_std = np.std(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_std'] = altitude_std
        altitude_max = np.max(altitudes_list) if altitudes_list else 0
        # row[f'{part}@altitude_max'] = altitude_max
        altitude_min = np.min(altitudes_list) if altitudes_list else 0
        # row[f'{part}@altitude_min'] = altitude_min
        altitude_range = altitude_max - altitude_min
        row[f'{part}@altitude_range'] = altitude_range
        altitude_cov = np.std(altitudes_list) / np.mean(altitudes_list) if altitudes_list else 0
        row[f'{part}@altitude_cov'] = altitude_cov

        # 고도 다양성
        hist_alt, _ = np.histogram(altitudes_list, bins=10, density=True)
        alt_ent = entropy(hist_alt) if hist_alt.sum() > 0 else 0
        row[f'{part}@altitude_entropy'] = alt_ent

        # 속도 기초 통계량
        speed_mean = np.mean(speeds_list) if speeds_list else 0
        row[f'{part}@speed_mean'] = speed_mean
        speed_max = np.max(speeds_list) if speeds_list else 0
        row[f'{part}@speed_max'] = speed_max
        speed_std = np.std(speeds_list) if speeds_list else 0
        row[f'{part}@speed_std'] = speed_std
        speed_cov =   np.std(speeds_list) / np.mean(speeds_list) if speeds_list else 0
        row[f'{part}@speed_cov'] = speed_cov

        # 정지 속도 수
        row[f'{part}@speed_zero_count'] =  np.sum([s == 0 for s in speeds_list]) / len(speeds_list) if speeds_list else 0

        # 속도 다양성
        hist_spd, _ = np.histogram(speeds_list, bins=10, density=True)
        speed_ent = entropy(hist_spd) if hist_spd.sum() > 0 else 0
        row[f'{part}@speed_entropy'] = speed_ent

        # 위도 경도 요약
        longitude_std = np.std(longitudes_list) if longitudes_list else 0
        row[f'{part}@longitude_std'] = longitude_std
        latitude_std = np.std(latitudes_list) if latitudes_list else 0
        row[f'{part}@latitude_std'] = latitude_std

        # mgps 불안전성
        row[f'{part}@mgps_instability'] = ((1 + speed_ent) * (1+ alt_ent) * (1 + abs(1 - path_tortuosity)) *
         (1 + speed_cov) * (1 + altitude_cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mGps_df = preprocessing_mGps(mGps_df)

# def cummean_mGps(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mGps_df = cummean_mGps(pre_mGps_df)

# def final_mGps(pre_mGps_df, cummean_mGps_df):
#   merge_df = pre_mGps_df.merge(cummean_mGps_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mGps_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mGps_df = final_mGps(pre_mGps_df, cummean_mGps_df)

In [ ]:
pre_mGps_df

,subject_id,date,S1_S2@center_from_max_distance,S1_S2@center_total_distance,S1_S2@total_distance,S1_S2@path_tortuosity,S1_S2@altitude_mean,S1_S2@altitude_std,S1_S2@altitude_range,S1_S2@altitude_cov,S1_S2@altitude_entropy,S1_S2@speed_mean,S1_S2@speed_max,S1_S2@speed_std,S1_S2@speed_cov,S1_S2@speed_zero_count,S1_S2@speed_entropy,S1_S2@longitude_std,S1_S2@latitude_std,S1_S2@mgps_instability
0,id01,2024-06-26,4.885207,1750.182969,13.335242,131.244933,101.479539,5.953680,38.3,0.058669,0.450268,0.449765,32.4100,2.108523,4.688056,0.471219,0.125670,0.008764,0.001407,1290.226446
1,id01,2024-06-27,4.884633,1814.970451,9.753022,186.093143,100.032185,6.184678,41.5,0.061827,0.982334,0.288264,32.5100,2.052651,7.120728,0.629695,0.111086,0.008742,0.001380,3534.301115
2,id01,2024-06-28,0.300786,7.749181,7.512160,1.031552,103.855739,1.123436,31.9,0.010817,0.014327,0.290184,21.2428,0.643166,2.216405,0.005515,0.033382,0.000066,0.000063,3.515390
3,id01,2024-06-29,0.009887,1.281903,0.705337,1.817435,119.404227,8.177879,30.0,0.068489,1.017719,0.019202,3.2589,0.095499,4.973320,0.920205,0.105611,0.000006,0.000003,25.876643
4,id01,2024-06-30,4.834951,2212.954320,9.467698,233.737322,109.710783,9.981166,54.6,0.090977,1.074462,0.281262,33.0094,2.093341,7.442684,0.731643,0.110164,0.009645,0.001630,4958.119170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,id10,2024-09-22,0.024220,0.743136,0.151368,4.909477,99.895367,0.096471,2.2,0.000966,0.017845,0.012156,1.5356,0.074575,6.134812,0.475676,0.129517,0.000008,0.000008,40.309844
755,id10,2024-09-23,0.047731,2.888208,3.355845,0.860650,107.999546,0.028571,1.8,0.000265,0.002340,0.135259,5.1381,0.137097,1.013589,0.003780,0.012293,0.000007,0.000008,2.328437
756,id10,2024-09-24,0.016500,5.528785,1.989021,2.779651,99.900507,0.085087,4.4,0.000852,0.027109,0.077131,2.5409,0.182205,2.362285,0.293600,0.422468,0.000015,0.000007,13.666381
757,id10,2024-09-25,0.012975,10.408144,2.249476,4.626919,99.899592,0.098458,7.8,0.000986,0.008552,0.072684,1.7653,0.138100,1.900006,0.135604,0.484495,0.000020,0.000006,20.109246


### mLight

In [ ]:
def preprocessing_mLight(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (sub, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': sub, 'date': date}
      for time_label, block_group in group.groupby('time_part'):
        light_list = block_group[col].tolist()

        # mlight 기초 통계
        mean = np.mean(light_list)
        row[f'{time_label}@mlight_mean'] = mean
        std = np.std(light_list)
        row[f'{time_label}@mlight_std'] = std
        cov = np.std(light_list) / np.mean(light_list)
        row[f'{time_label}@mlight_cov'] = cov
        max = np.max(light_list)
        # row[f'{time_label}@mlight_max'] = max
        min = np.min(light_list)
        # row[f'{time_label}@mlight_min'] = min
        range = max - min
        row[f'{time_label}@mlight_range'] = range

        # might 고유값
        light_cnt = len(set(light_list)) / len(light_list)
        row[f'{time_label}@mlight_cnt'] = light_cnt

        # mlight 다양성
        hist_light, _ = np.histogram(light_list, bins=10, density=True)
        ent = entropy(hist_light) if hist_light.sum() > 0 else 0
        row[f'{time_label}@mlight_entropy'] = ent

        # mlight 불안전성
        row[f'{time_label}@mlight_instability'] = (1 + ent) * (1 + cov) * (1 + light_cnt)

      results.append(row)

    return pd.DataFrame(results)

pre_mLight_df = preprocessing_mLight(mLight_df)

# def cummean_mLight(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mLight_df = cummean_mLight(pre_mLight_df)

# def final_mLight(pre_mLight_df, cummean_mLight_df):
#   merge_df = pre_mLight_df.merge(cummean_mLight_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mLight_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mLight_df = final_mLight(pre_mLight_df, cummean_mLight_df)

In [ ]:
pre_mLight_df

,subject_id,date,S1_S2@mlight_mean,S1_S2@mlight_std,S1_S2@mlight_cov,S1_S2@mlight_range,S1_S2@mlight_cnt,S1_S2@mlight_entropy,S1_S2@mlight_instability
0,id01,2024-06-26,59.346939,118.350491,1.994214,373.000000,0.285714,0.995458,7.681924
1,id01,2024-06-27,34.416667,97.797587,2.841576,360.000000,0.166667,0.485216,6.656497
2,id01,2024-06-28,28.375000,73.889733,2.604043,273.000000,0.229167,0.641429,7.271483
3,id01,2024-06-29,0.166667,0.874007,5.244044,6.000000,0.062500,0.273574,8.449267
4,id01,2024-06-30,23.208333,80.955996,3.488230,345.000000,0.125000,0.373479,6.935052
...,...,...,...,...,...,...,...,...,...
797,id10,2024-09-22,32.242501,51.841878,1.607874,202.972504,0.833333,0.887694,9.025259
798,id10,2024-09-23,1.833750,10.848611,5.916080,66.014999,0.055556,0.126931,8.226938
799,id10,2024-09-24,89.647034,120.478363,1.343919,305.977509,0.479167,1.404921,8.337973
800,id10,2024-09-25,49.665939,75.670265,1.523585,348.975006,0.458333,1.326743,8.562943


### mScreenStatus

In [ ]:
def preprocessing_mScreenStatus(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []
    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):
        status = part_group[col].values

        # 총 화면 켜진 기간
        # on_screen_period = sum(status)
        # row[f'{part}on_screen_period'] = on_screen_period

        # 화면 켜진상태 비율
        on_screen_ratio = status.mean()
        row[f'{part}@on_screen_ratio'] = on_screen_ratio

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum() / len(status)
        row[f'{part}@on_screen_transitions'] = transitions

        # 사용 패턴 분포
        _, counts = np.unique(status, return_counts=True)
        prob = counts / counts.sum()
        ent = entropy(prob) # 1에 가까울수록 사용 상태가 자주 바뀜
        row[f'{part}@on_screen_entropy'] = ent

        # 연속된 1 상태 길이
        length = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                length.append(current_len)
                current_len = 0
        # 배열이 1로 끝날 때 마지막 구간 길이
        if current_len > 0:
            length.append(current_len)

        # 화면 켜진 기간 기초통계량
        avg_on_screen = np.mean(length) if length else 0
        row[f'{part}@avg_on_screen'] = avg_on_screen
        std_on_screen = np.std(length) if length else 0
        row[f'{part}@std_on_screen'] = std_on_screen
        cov_on_screen = (np.std(length) / np.mean(length)) if length else 0
        row[f'{part}@cov_on_screed'] = cov_on_screen
        max_on_screen = np.max(length) if length else 0
        # row[f'{part}@max_on_screen'] = max_on_screen
        min_on_screen = np.min(length) if length else 0
        # row[f'{part}@min_on_screen'] = min_on_screen
        range_on_screen = max_on_screen - min_on_screen
        row[f'{part}@range_on_screen'] = range_on_screen

        # 화면 불안전성
        row[f'{part}@mscreen_instability'] = (1 + ent) * (1 + transitions) * (1 + on_screen_ratio) * (1+ cov_on_screen)

      results.append(row)

    return pd.DataFrame(results)

pre_mScreenStatus_df = preprocessing_mScreenStatus(mScreenStatus_df)

# def cummean_mScreenStatus(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mScreenStatus_df = cummean_mScreenStatus(pre_mScreenStatus_df)

# def final_mScreenStatus(pre_mScreenStatus_df, cummean_mScreenStatus_df):
#   merge_df = pre_mScreenStatus_df.merge(cummean_mScreenStatus_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mScreenStatus_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mScreenStatus_df = final_mScreenStatus(pre_mScreenStatus_df, cummean_mScreenStatus_df)

In [ ]:
pre_mScreenStatus_df

,subject_id,date,S1_S2@on_screen_ratio,S1_S2@on_screen_transitions,S1_S2@on_screen_entropy,S1_S2@avg_on_screen,S1_S2@std_on_screen,S1_S2@cov_on_screed,S1_S2@range_on_screen,S1_S2@mscreen_instability
0,id01,2024-06-26,0.153017,0.015086,0.427907,17.750000,24.437420,1.376756,58.0,3.972129
1,id01,2024-06-27,0.163830,0.010638,0.445966,25.666667,29.959232,1.167243,65.0,3.685963
2,id01,2024-06-28,0.079167,0.006250,0.276729,19.000000,15.000000,0.789474,30.0,2.480953
3,id01,2024-06-29,0.008696,0.002174,0.049918,4.000000,0.000000,0.000000,0.0,1.061350
4,id01,2024-06-30,0.087500,0.018750,0.296715,8.400000,7.657676,0.911628,19.0,2.746281
...,...,...,...,...,...,...,...,...,...,...
798,id10,2024-09-22,0.945455,0.036364,0.211688,34.666667,35.705586,1.029969,79.0,4.959219
799,id10,2024-09-23,0.053412,0.011869,0.208443,9.000000,8.000000,0.888889,16.0,2.433075
800,id10,2024-09-24,0.523404,0.044681,0.692051,22.363636,42.002755,1.878172,126.0,7.750488
801,id10,2024-09-25,0.544444,0.037778,0.689191,27.222222,41.659407,1.530346,140.0,6.850706


### mUsageStats

In [ ]:
def preprocessing_mUsageStats(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': subj, 'date': date}
      for part, part_group in group.groupby('time_part'):
        all_labels_scores = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          total_time_sum = sum(item.get('total_time', 0) for item in flat if isinstance(item, dict))
          for item in flat:
            label = item['app_name']
            score = float(item['total_time'])
            if total_time_sum > 0:
              normalized_score = score / total_time_sum
              all_labels_scores.append((label, normalized_score))

          if all_labels_scores:
            label_list = [label for label, _ in all_labels_scores]
            score_list = [score for _, score in all_labels_scores]
            labels, scores = zip(*all_labels_scores)
            scores = list(scores)
            max_idx = np.argmax(scores)
            min_idx = np.argmin(scores)

            # 사용 최대 라벨
            # row[f'{part}@usagestats_max_label'] = labels[max_idx]

            # 사용 최소 라벨
            # row[f'{part}@usagestats_min_label'] = labels[min_idx]

            # 사용 범위
            range = scores[max_idx] - scores[min_idx]
            row[f'{part}@usagestats_range'] = range

            # 사용 고유값
            usagestats_cnt = len(set(label_list)) / len(label_list)
            row[f'{part}@usagestats_cnt'] = usagestats_cnt

            # 사용 형태 다양성 분포
            probs_counter = Counter(label_list)
            total = sum(probs_counter.values())
            probs = [v / total for v in probs_counter.values()]
            ent = entropy(probs)
            row[f'{part}@usagestats_entropy'] = ent

            # 기초 통계량
            mean = np.mean(scores) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_mean'] = mean
            std = np.std(scores) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_std'] = std
            cov = (np.std(scores) / np.mean(scores)) if sum(scores) > 0 else 0
            row[f'{part}@usagestats_cov'] = cov

            # 사용 불안전성
            row[f'{part}@usagestats_instability'] = (1 + ent) * (1 + usagestats_cnt) * (1 + cov)

      results.append(row)

    return pd.DataFrame(results)

pre_mUsageStats_df = preprocessing_mUsageStats(mUsageStats_df)

# def cummean_mUsageStats(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mUsageStats_df = cummean_mUsageStats(pre_mUsageStats_df)

# def final_mUsageStats(pre_mUsageStats_df, cummean_mUsageStats_df):
#   merge_df = pre_mUsageStats_df.merge(cummean_mUsageStats_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mUsageStats_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mUsageStats_df = final_mUsageStats(pre_mUsageStats_df, cummean_mUsageStats_df)

In [ ]:
pre_mUsageStats_df

,subject_id,date,S1_S2@usagestats_range,S1_S2@usagestats_cnt,S1_S2@usagestats_entropy,S1_S2@usagestats_mean,S1_S2@usagestats_std,S1_S2@usagestats_cov,S1_S2@usagestats_instability
0,id01,2024-06-26,0.999787,0.325000,2.416505,0.225000,0.266711,1.185382,9.892937
1,id01,2024-06-27,0.994831,0.375000,2.724246,0.208333,0.287500,1.379999,12.187587
2,id01,2024-06-28,0.999397,0.454545,2.107426,0.363636,0.390260,1.073215,9.370709
3,id01,2024-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id01,2024-06-30,0.995698,0.450000,2.552679,0.200000,0.264462,1.322309,11.963105
...,...,...,...,...,...,...,...,...,...
785,id10,2024-09-22,0.995665,0.227273,1.453508,0.454545,0.417112,0.917647,5.774272
786,id10,2024-09-23,0.984949,0.571429,1.945910,0.285714,0.278168,0.973590,9.136314
787,id10,2024-09-24,0.999899,0.216495,2.388252,0.309278,0.311858,1.008342,8.277968
788,id10,2024-09-25,0.999791,0.208333,2.413263,0.322917,0.346664,1.073539,8.552020


### mwifi

In [ ]:
def preprocessing_mwifi(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id':subj, 'date':date}
      for part, part_group in group.groupby('time_part'):

        rssi_list = []
        bssid_list = []

        for device in part_group[col]:
          flat = ast.literal_eval(device) if isinstance(device, str) else device
          for device in flat:

              # rssi 목록
              rssi = int(device.get('rssi'))
              rssi_list.append(rssi)

              # bssid 목록
              bssid = str(device.get('bssid'))
              bssid_list.append(bssid)

        # rssi 고유값
        rssi_cnt = len(set(rssi_list)) / len(rssi_list)
        row[f'{part}@rssi_cnt'] = rssi_cnt

        # rssi 다양성
        rssi_counter = Counter(rssi_list)
        total = sum(rssi_counter.values())
        probs = [v / total for v in rssi_counter.values()]
        ent_rssi = entropy(probs)
        row[f'{part}@rssi_entropy'] = ent_rssi

        # bssid 고유값
        bssid_cnt = len(set(bssid_list)) / len(bssid_list)
        row[f'{part}@bssid_cnt'] = bssid_cnt

        # bssid 다양성
        bssid_counter = Counter(bssid_list)
        total = sum(bssid_counter.values())
        probs = [v / total for v in bssid_counter.values()]
        ent_bssid = entropy(probs)
        row[f'{part}@bssid_entropy'] = ent_bssid

        # 알수없는 rssi 비율
        weak_rssi = [r for r in rssi_list if r < -100 or r > - 25]
        rssi_Uncommon_cnt = len(weak_rssi)
        rssi_Uncommon_ratio =  rssi_Uncommon_cnt / len(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_Uncommon_ratio'] = rssi_Uncommon_ratio

        # 기초 통계
        mean = np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_mean'] = np.mean(rssi_list) if rssi_list else 0
        min = np.min(rssi_list) if rssi_list else 0
        max = np.max(rssi_list) if rssi_list else 0
        range = np.max(rssi_list) - np.min(rssi_list)
        row[f'{part}@rssi_range'] = range
        std = np.std(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_std'] = std
        cov = np.std(rssi_list) / np.mean(rssi_list) if rssi_list else 0
        row[f'{part}@rssi_cov'] = cov

        # mble 불안전성
        row[f'{part}@wifi_instability'] = ((1 + ent_rssi) * (1 + rssi_cnt) *
         (1 + ent_bssid) * (1 + bssid_cnt) *
           (1 + cov))

      results.append(row)

    return pd.DataFrame(results)

pre_mWifi_df = preprocessing_mwifi(mWifi_df)

# def cummean_mWifi(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_mWifi_df = cummean_mWifi(pre_mWifi_df)

# def final_mWifi(pre_mWifi_df, cummean_mWifi_df):
#   merge_df = pre_mWifi_df.merge(cummean_mWifi_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_mWifi_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_mWifi_df = final_mWifi(pre_mWifi_df, cummean_mWifi_df)

In [ ]:
pre_mWifi_df

,subject_id,date,S1_S2@rssi_cnt,S1_S2@rssi_entropy,S1_S2@bssid_cnt,S1_S2@bssid_entropy,S1_S2@rssi_Uncommon_ratio,S1_S2@rssi_mean,S1_S2@rssi_range,S1_S2@rssi_std,S1_S2@rssi_cov,S1_S2@wifi_instability
0,id01,2024-06-26,0.158537,3.635970,0.225610,3.611008,0.00000,-65.496951,62.0,19.174583,-0.292755,21.466838
1,id01,2024-06-27,0.196296,3.676703,0.266667,3.679875,0.00000,-65.777778,62.0,17.861470,-0.271543,24.159020
2,id01,2024-06-28,0.125714,3.270199,0.100000,2.597302,0.00000,-58.431429,62.0,21.376078,-0.365832,12.062860
3,id01,2024-06-29,0.432836,3.170846,0.223881,2.385876,0.00000,-62.328358,56.0,19.769106,-0.317177,16.909820
4,id01,2024-06-30,0.254630,3.722414,0.287037,3.657122,0.00463,-64.625000,65.0,17.536816,-0.271363,25.876150
...,...,...,...,...,...,...,...,...,...,...,...,...
780,id10,2024-09-22,0.320896,3.499297,0.231343,3.084748,0.00000,-70.947761,50.0,12.417679,-0.175026,24.660277
781,id10,2024-09-23,0.070941,3.508782,0.085948,3.633649,0.00000,-65.624829,55.0,15.954567,-0.243118,18.390158
782,id10,2024-09-24,0.079745,3.398371,0.106858,3.577953,0.00000,-73.062201,52.0,10.932778,-0.149637,20.463545
783,id10,2024-09-25,0.082963,3.464517,0.111111,3.629105,0.00000,-72.650370,56.0,11.623315,-0.159990,20.889459


### wHr

In [ ]:
def preprocessing_wHr(dfs):
  df = sleep_lifelog(dfs)
  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
  col = find_col(df)

  results = []

  for (sub, date), group in df.groupby(['subject_id', 'date']):
    row = {'subject_id':sub, 'date': date}

    for part, part_group in group.groupby('time_part'):
      hr_list = []
      for entry in part_group[col]:
        flat = ast.literal_eval(entry) if isinstance(entry, str) else entry
        hr_list.extend([int(hr) for hr in flat if hr is not None])

      # 심장박동 기초 통계량
      mean = np.mean(hr_list) if hr_list else 0
      row[f'{part}@hr_mean'] = mean
      std = np.std(hr_list) if hr_list else 0
      row[f'{part}@hr_std'] = std
      max = np.max(hr_list) if hr_list else 0
      # row[f'{part}@hr_max'] = max
      min = np.min(hr_list) if hr_list else 0
      # row[f'{part}@hr_min'] = min
      range = max - min
      row[f'{part}@hr_range'] = range
      cov = (np.std(hr_list) / np.mean(hr_list)) if hr_list else 0
      row[f'{part}@hr_cov'] = cov

      # 비정상적인 박동
      hr_over = len([hr for hr in hr_list if hr >= 100]) / len(hr_list)
      # row[f'{part}@hr_100_over_cnt'] = hr_over
      hr_under = len([hr for hr in hr_list if hr <= 60]) / len(hr_list)
      # row[f'{part}@hr_60_under_cnt'] = hr_under
      hr_abnormal = hr_over + hr_under
      row[f'{part}@hr_abnormal_cnt'] = hr_abnormal

      # 심장박동 다양성 분포
      hist, _ = np.histogram(hr_list, bins=12, range=(40, 160), density=True)
      ent = entropy(hist)
      row[f'{part}@hr_entropy'] = ent

      # 심장박동 불안전성
      row[f'{part}@hr_instability'] = (1 + ent) * (1 + cov) * (1 + hr_abnormal)

    results.append(row)

  return pd.DataFrame(results)

pre_wHr_df = preprocessing_wHr(wHr_df)

# def cummean_wHr(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_wHr_df = cummean_wHr(pre_wHr_df)

# def final_wHr(pre_wHr_df, cummean_wHr_df):
#   merge_df = pre_wHr_df.merge(cummean_wHr_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_wHr_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)
#   return final_df

# final_wHr_df = final_wHr(pre_wHr_df, cummean_wHr_df)

In [ ]:
pre_wHr_df

,subject_id,date,S1_S2@hr_mean,S1_S2@hr_std,S1_S2@hr_range,S1_S2@hr_cov,S1_S2@hr_abnormal_cnt,S1_S2@hr_entropy,S1_S2@hr_instability
0,id01,2024-06-26,86.170431,11.875367,47.0,0.137813,0.166324,1.561826,3.399693
1,id01,2024-06-27,90.841850,10.696596,54.0,0.117750,0.232383,1.468879,3.400869
2,id01,2024-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,id01,2024-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id01,2024-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
681,id10,2024-09-22,90.967132,6.451679,62.0,0.070923,0.060333,0.932441,2.194356
682,id10,2024-09-23,151.590021,18.665761,124.0,0.123133,0.999757,1.533667,5.690599
683,id10,2024-09-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684,id10,2024-09-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### wLight

In [ ]:
def preprocessing_wLight(dfs):
    df = sleep_lifelog(dfs)
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['date'] = df['lifelog_date'].dt.date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(['subject_id', 'timestamp'])
    df['time_part'] = df['timestamp'].dt.hour.map(cut_time)
    col = find_col(df)

    results = []

    for (sub, date), group in df.groupby(['subject_id', 'date']):
      row = {'subject_id': sub, 'date': date}
      for time_label, block_group in group.groupby('time_part'):
        light_list = block_group[col].tolist()

        # wLight 기초 통계
        mean = np.mean(light_list)
        row[f'{time_label}@wLight_mean'] = mean
        std = np.std(light_list)
        row[f'{time_label}@wLight_std'] = std
        cov = np.std(light_list) / np.mean(light_list)
        row[f'{time_label}@wLight_cov'] = cov
        max = np.max(light_list)
        # row[f'{time_label}@wLight_max'] = max
        min = np.min(light_list)
        # row[f'{time_label}@wLight_min'] = min
        range = max - min
        row[f'{time_label}@wLight_range'] = range

        # might 고유값
        light_cnt = len(set(light_list)) / len(light_list)
        row[f'{time_label}@wLight_cnt'] = light_cnt

        # wLight 다양성
        hist_light, _ = np.histogram(light_list, bins=10, density=True)
        ent = entropy(hist_light) if hist_light.sum() > 0 else 0
        row[f'{time_label}@wLight_entropy'] = ent

        # wLight 불안전성
        row[f'{time_label}@wLight_instability'] = (1 + ent) * (1 + cov) * (1 + light_cnt)

      results.append(row)

    return pd.DataFrame(results)

pre_wLight_df = preprocessing_wLight(wLight_df)

# def cummean_wLight(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_wLight_df = cummean_wLight(pre_wLight_df)

# def final_wLight(pre_wLight_df, cummean_wLight_df):
#   merge_df = pre_wLight_df.merge(cummean_wLight_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_wLight_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_wLight_df = final_wLight(pre_wLight_df, cummean_wLight_df)

In [ ]:
pre_wLight_df

,subject_id,date,S1_S2@wLight_mean,S1_S2@wLight_std,S1_S2@wLight_cov,S1_S2@wLight_range,S1_S2@wLight_cnt,S1_S2@wLight_entropy,S1_S2@wLight_instability
0,id01,2024-06-26,50.445455,89.112910,1.766520,282.0,0.122727,1.135318,6.632398
1,id01,2024-06-27,84.957672,124.020472,1.459791,305.0,0.116402,1.122027,5.827331
2,id01,2024-06-28,45.663551,105.112840,2.301898,311.0,0.056075,0.495353,5.214372
3,id01,2024-06-29,0.000000,0.000000,NaN,0.0,0.031250,0.000000,NaN
4,id01,2024-06-30,26.762500,86.427172,3.229413,311.0,0.037500,0.296715,5.690008
...,...,...,...,...,...,...,...,...,...
745,id10,2024-09-22,33.268908,37.009128,1.112424,184.0,0.168067,1.401174,5.924785
746,id10,2024-09-23,19.276730,12.351718,0.640758,40.0,0.037736,1.116469,3.603655
747,id10,2024-09-24,0.000000,0.000000,NaN,0.0,0.002660,0.000000,NaN
748,id10,2024-09-25,44.206061,36.347814,0.822236,80.0,0.012121,0.794064,3.308834


### wPedo

In [ ]:
def preprocessing_wPedo(dfs):
  df = sleep_lifelog(dfs)
  df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
  df['date'] = df['lifelog_date'].dt.date
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df = df.sort_values(['subject_id', 'timestamp'])
  df['time_part'] = df['timestamp'].dt.hour.map(cut_time)

  results = []

  for (subj, date), group in df.groupby(['subject_id', 'date']):
    row = {'subject_id':subj, 'date':date}
    for part, part_group in group.groupby('time_part'):
      step_list = []
      step_frequency_list = []
      distance_list = []
      speed_list = []
      burned_calories_list = []

      for _, entry in part_group.iterrows():
        step_list.append(entry['step'])
        step_frequency_list.append(entry['step_frequency'])
        distance_list.append(entry['distance'])
        speed_list.append(entry['speed'])
        burned_calories_list.append(entry['burned_calories'])

      # step 기초통계량
      step_sum = (np.sum(step_list) / len(step_list)) if sum(step_list) > 0 else 0
      row[f'{part}@step_sum'] = step_sum
      step_mean = np.mean(step_list) if sum(step_list) > 0 else 0
      row[f'{part}@step_mean'] = step_mean
      step_std = np.std(step_list) if sum(step_list) > 0 else 0
      row[f'{part}@step_std'] = step_std
      step_cov = (np.std(step_list) / np.mean(step_list)) if sum(step_list) > 0 else 0
      row[f'{part}@step_cov'] = step_cov

      # step 전이 수
      step_arr = np.array(step_list)
      transitions = np.sum(step_arr[1:] != step_arr[:-1]) / len(step_arr)
      row[f'{part}@step_transition'] = transitions

      # step 다양성
      hist, _ = np.histogram(step_arr, bins=10, density=True)
      step_entropy = entropy(hist) if hist.sum() > 0 else 0
      row[f'{part}@step_entropy'] = step_entropy

      # 보속 평균
      sf_mean = np.mean(step_frequency_list) if np.sum(step_frequency_list) > 0 else 0
      row[f'{part}@step_frequency_mean'] = sf_mean

      # 총 이동거리
      d_sum = np.sum(distance_list) / len(distance_list)
      row[f'{part}@distance_sum'] = d_sum

      # 속도 기초통계량
      speed_mean = np.mean(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_mean'] = speed_mean
      speed_std = np.std(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_std'] = speed_std
      speed_cov = (np.std(speed_list) / np.mean(speed_list)) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_cov'] = speed_std
      speed_max = np.max(speed_list) if speed_list and np.sum(speed_list) > 0 else 0
      row[f'{part}@pedo_speed_max'] = speed_max

      # 총 소비된 칼로리
      bc_sum = (np.sum(burned_calories_list) / len(burned_calories_list)) if burned_calories_list and np.sum(burned_calories_list) > 0 else 0
      row[f'{part}@burned_calories_sum'] = bc_sum

      # 추정 보폭 평균
      if step_list and distance_list:
        step_sum = np.sum(step_list)
        if step_sum > 0:
          stride_mean = np.sum(distance_list) / step_sum
        else:
          stride_mean = 0
      else:
          stride_mean = 0
      row[f'{part}@stride_mean'] = stride_mean

      # pedo 불안전성
      instability = ((1 + step_entropy) * (1 + step_cov) *
       (1 + transitions) *
        (1 + speed_cov) * (1 + stride_mean))
      row[f'{part}@pedo_instability'] = instability

    results.append(row)

  return pd.DataFrame(results)

pre_wPedo_df = preprocessing_wPedo(wPedo_df)

# def cummean_wPedo(df):
#     mean_df = pd.DataFrame()

#     num_col = df.select_dtypes(include='number').columns
#     for col in num_col:
#       # mean_df[f'{col}_cummean'] = (df.groupby('subject_id')[col].expanding().mean())
#       mean_df[f'{col}'] = (df.groupby('subject_id')[col].mean())

#     return mean_df.reset_index()

# cummean_wPedo_df = cummean_wPedo(pre_wPedo_df)

# def final_wPedo(pre_wPedo_df, cummean_wPedo_df):
#   merge_df = pre_wPedo_df.merge(cummean_wPedo_df, on='subject_id', suffixes=('', '_cummean'))

#   final_df = pd.DataFrame()
#   final_df['subject_id'] = merge_df['subject_id']
#   final_df['date'] = merge_df['date']

#   for col in pre_wPedo_df.columns:
#     if '@' in col:
#       cummean_col = f'{col}_cummean'
#       if 'S1_S2' in col:
#         final_df[f'{col}'] = (merge_df[col] < merge_df[cummean_col]).astype(int)

#   return final_df

# final_wPedo_df = final_wPedo(pre_wPedo_df, cummean_wPedo_df)

In [ ]:
pre_wPedo_df

,subject_id,date,S1_S2@step_sum,S1_S2@step_mean,S1_S2@step_std,S1_S2@step_cov,S1_S2@step_transition,S1_S2@step_entropy,S1_S2@step_frequency_mean,S1_S2@distance_sum,S1_S2@pedo_speed_mean,S1_S2@pedo_speed_std,S1_S2@pedo_speed_cov,S1_S2@pedo_speed_max,S1_S2@burned_calories_sum,S1_S2@stride_mean,S1_S2@pedo_instability
0,id01,2024-06-26,0.542299,0.542299,5.289130,9.753155,0.028200,0.106183,0.009038,0.418112,0.006969,0.069395,0.069395,1.290666,0.009935,0.770999,237.357368
1,id01,2024-06-27,0.504167,0.504167,5.474180,10.857878,0.020833,0.079839,0.008403,0.392708,0.006545,0.071552,0.071552,1.234000,0.016208,0.778925,277.456973
2,id01,2024-06-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,id01,2024-06-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,id01,2024-06-30,0.272251,0.272251,3.931909,14.442205,0.010471,0.054514,0.004538,0.220236,0.003671,0.054604,0.054604,1.037333,0.002644,0.808942,472.559480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,id10,2024-09-21,0.370833,0.370833,2.477394,6.680614,0.052083,0.164657,0.006181,0.264374,0.004406,0.029561,0.029561,0.431661,0.016438,0.712920,124.271382
730,id10,2024-09-22,0.420561,0.420561,2.867926,6.819290,0.056075,0.158655,0.007009,0.307480,0.005125,0.034900,0.034900,0.279842,0.024861,0.731120,129.363041
731,id10,2024-09-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
732,id10,2024-09-25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


# 데이터 합치기

In [ ]:
dfs_list = []
dfs = dataframe.copy()
for k, v in dfs.items():
  pre_name = f'pre_{k}_df'
  dfs_list.append(pre_name)
  print(pre_name)

# pre_list = {'pre_' + k + '_df2': v for k, v in dfs.items()}
# pre_name1

pre_mACStatus_df
pre_mActivity_df
pre_mAmbience_df
pre_mBle_df
pre_mGps_df
pre_mLight_df
pre_mScreenStatus_df
pre_mUsageStats_df
pre_mWifi_df
pre_wHr_df
pre_wLight_df
pre_wPedo_df


In [ ]:
df_list = [pre_mACStatus_df,
pre_mActivity_df,
pre_mAmbience_df,
pre_mBle_df,
pre_mGps_df,
pre_mLight_df,
pre_mScreenStatus_df,
pre_mUsageStats_df,
pre_mWifi_df,
pre_wHr_df,
pre_wLight_df,
pre_wPedo_df]

In [ ]:
prep_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list)

In [ ]:
# 전처리된 데이터 저장
prep_df.to_csv('/content/drive/MyDrive/1데이콘/ETRI_휴먼이해_인공지능/ETRI_lifelog_dataset/prep_df_S1-2.csv',index=False)